# CNN的网络结构 



## Outline 
### 历史上的经典结构类型
- 开山(已淘汰)
    - 史前
    - 1998 LeNet
        - 只有5层, 也叫LeNet5
    - 2012 AlexNet
        - 在历史的发展中很重要, 为其他框架提供了借鉴意义
        - 最早开始有CAFFE框架
        - GPU普及, 算力提升
        - 转折点, DP开始火爆
    - 2013 ZFNet
        - 可视化分析
- 二代(仍用)
    - 2014 VGG(Visual Geometry Group) LeNet
        - 很好的特征提取网络
        - 比较深
    - 2014 GoogLeNet(inception)
        - 模块化, 积木化
- 三代
    - 2015 ResNet
        - 更深(成百上千)
    - 模型的轻量化
        - SqueezeNet->MobileNet -> ShuffleNet
    - 2017 DenseNet
    - 2017 CapsNet
    
### 模型的参数
- 参数多少: FLOPs, 能衡量模型的大小,
- 计算次数: MACs计算次数

### 模型的结构
- 模型都是由backbone(提取特征,加以组合, 也就是backbone就是特征处理部分)得到feature map,
- 然后功能层(或是分类,或是检测)对fm做处理得结果

### 分类模型正确率的提高
- 2011 XRCE 74%
- 2012 AlexNet 83.6%
- 2013 ZFNet 88.3%
- 2014 VGG 92.7%
- 2014 GoogLeNet 93.3%
- **human** 95%
- 2015 ResNet 96.4%
- 2016 GoogLeNet - v4 96.9%

## 1998 LeNet

3@28x28 -> Conv 5x5x3x20 -> tanh -> 20@24x24 -> max-pooling 2x2 -> 20@12x12 -> Conv 5x5x20x50 -> tanh -> 50@8x8 -> max-pooling 2x2 -> 50@4x4 -> Conv -> tanh -> Flatten -> 500 -> FC ->tanh


## AlexNet
3@224x224 -> Conv 11x11x3x96 stride = 4 -> 55x55x96 ->  Conv 5x5x96x256 stride = 1 -> max-pooling 2x2 -> LRN -> 27x27x256 -> Conv 3x3x256x384 -> max-pooling 2x2 -> LRN -> 13x13x384 -> Conv 3x3x384x384 -> 13x13x384 -> Conv 3x3x384x256 -> 13x13x256 -> max-pooling -> Flatten -> FC

- 用了Relu
- 用了Dropout
- 用了重叠的max pooling
    - 比如3x3的窗口,每2个遍历一次, 这样就会有重叠的
- 用了多个GPU计算
- LRN Local Response Normalization技术（一种归一化技术，已被淘汰）

## ZFNet



## 小结

在这一历史时期，这一经验公式empirical formula为核心：

Input -> n x (Conv - ReLU - MaxPooling) -> 1|2 x FC -> Output

刚起步阶段，其中很多东西都可以改进


## VGG
- 卷积核进一步缩小
- 连续卷积再pooling，而不是卷积pooling
- pooling 2x2 stride =2 
- 通道很多
- 一般是16层/19层
- 只用3x3的卷积核
    - 最早倾向去前面的layer用大核，后面用小核，大核的感受野大，提取的特征区域也大
    - 而小核用多次，也能实现更大区域的特征。例如两次3x3就是一次5x5的，三次3x3就是一次7x7的
    - 三次3x3需要27x原图channel数个参数，而一次7x7需要49x原图channel数个参数，节省空间
    - 小核和大核有一样的RF加速算法，小核对Winograd加速算法友好
    - 注意channel会变化，比如channel比较少，大核也可以

## GoogLeNet

- 启发于盗梦空间，很深，很宽
- 其中一个模块重复了很多次
    - 前一层有四个分支，分别做卷积，然后做depthConcat，也就是图像大小一样，张数叠加
    - 用这四个分支，获得感受野的区域不一样，我们把这四个不同感受野放在一起，特征更加丰富
- 3个输出
- 没有FC(初代有)
- Global ave pooling
    - 比如500个特征图, 每个图7x7, 在7x7中取一个平均值,得到500个均值
    - 减少了Fc操作
- auxiliary softmax 辅助的
    - 有三个输出, 每个输出都和真实值做loss, 然后梯度回传
    - 三个模型的输出做一个ensemble, 也就是对loss有权值, 加权求和一起回传
    - 层数浅的, 梯度可以大,权重小,  收敛快
    - 减少过拟合
- 网络层数增加,参数也增加, 增强你和能力
- 借鉴了Hebbin's Principle, 多个神经元多次同时激活,  某次只激活一个,其他的也会按惯例激活

## ResNet
###  背景

归根结底,网络不够深, 不能拟合复杂的

设计的很深的时候, 比如56层对比5层, 和20层对比5层, 随着iteration增加, 虽然loss都降低

20层的确比5层的loss小,但是50的loss比20的大

每次回传的参数变化很小

梯度消失, 梯度下降法不太有效了

即便是relu,也只是缓解了梯度消失,层数多的时候还是会梯度消失

### 过程特点
- pooling次数很少
- 每个模块, 两个分支,一个做多层卷积,一个直接结果, 然后对应元素相加(size要一样),再激活
    - H(x) = F(x) + x这样求导不会小于1, 保证了梯度必定回传
    - F(x) = H(x) - x标签和输入的差,也叫残差,所以也叫残差网络
- bottle neck先降维再升维(瓶颈形)
- 一开始第一层7x7
    - 靠前的layer, channel 少的时候用大核, 尽量多地提取特征
    - ![10](10.png)
    - ![11](11.png)
- 虚线的地方跳阶,对应不跳阶的地方stride=2,因此"直接结果"的分支上需要加一个卷积;实线跳阶的地方对应stride=1

### 改进
- 更深
    - [Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf)
- 与Inception结合,32个分支, 宽度加宽.
    - [ResNext](https://arxiv.org/pdf/1611.05431.pdf)

### 为什么ResNet更好
多模型融合的优势, 就像电阻串并联, 可以排列组合出很多情况

![15](15.png)


## DenseNet
由ResNet发展而来, 的确更好,但是太花费开销了

神经元任意两个都要连接, 所有的组合

# 更轻量化的框架
2015 - 2016 backbone已经到头了, 模型的深度和宽度也没什么了

之后发展的趋势就是轻量化

比如华为2020 ghost net

两个问题:
- 模型的参数
- 计算的次数(加法 乘法)

解决:
- 模型本身
    - 剪枝, 量化(浮点变整型)
    - **结构**
        - 优化网络结构: ShuffleNet
        - 单纯减少参数: SqueezeNet
        - 优化卷积: 深度分离卷积, 组卷积
        - 删除FC, 用global pooling代替
- 外在方面
    - 设备(移动端/嵌入式)
    - 降低准度要求


## SqueezNet

- 来源于GoogLeNet
- 少用3x3,多用1x1,用3x3的时候channel少一点
- 为了防止参数太少, 晚点用pooling

### 结构过程
Fire Module(替代GoogLeNet中的那个块):

1. SqueezNet
    - 1x1 Conv 降维 成S1个通道
2. Expand
    - 两个分支 E1个1x1和E3个3x3
    - 深度concat

要求: 
- S1/(E1+E3) =0.75比较好
- E3/(E1+E3) 从0.5开始调

3种模型,第二种比较好:

![20](20.png)

## MobileNet V1
### 深度分离卷积:

1.Depthwise Convolution:

HxW的图,C个通道, 有C个KxKx1的卷积核, 一一对应做卷积(而不是全部)， 不叠加，还是得到C个结果

2.然后有N个1x1xC个卷积核,对上一步结果做普通卷积（c个通道对应卷积再叠加，重复N次）得到N个图

实质上相当于对于原来的KxKxCxN的卷积分开了，

#### 参数个数比较:

不考虑padding和stride

原来的方式:KxKxCxNxHxW=N·K^2·HWC 次遍历操作

深度分离卷积: KxKx1xCxHxW + 1x1xCxNxHxW = (N+ K^2) HWC 次操作

显然小了很多
### 过程与结构

![p23](23.png)

- 不能直接把深度分离卷积换成原来的, 还需要做一些修补

    - 原来的： 3x3 Conv -> BN -> ReLU

    - 现在：3x3 Depthwise Conv -> BN -> ReLU -> 1x1 Conv -> BN -> ReLU

- 全局ave pooling


- HyperParameter 超参数，也叫宽度因子alpha来控制瘦身程度， 分辨率因子\rho 控制特征图尺寸

$$K\times K\times 1\times C\times H\times W + 1\times 1\times C\times N\times H\times W $$
$$K\times K\times 1\times\alpha C\times\rho H\times \rho W + 1\times 1\times\alpha C\times\alpha N\times\rho H\times\rho W $$
$$\alpha \in (0,1],\;\;\rho \in (0,1]$$


Multiply-Accumulates乘法加法累积总数，简称MAC
![24](24.png)

### 一些细节
- 不太用于过拟合
- 不用太多数据增广
- 取消了L2正则化

## MoblieNet V2
两个改进：（借鉴于ResNet）
- ### Inverted Residuals 
    Skip connection这种瓶颈结构很有效

    但是参数再压缩就没了

    和ResNet类似，也是两个分支,一个做多层卷积,一个直接结果, 然后对应元素相加

    但是这个多层卷积不是channel先变小在变大，而是先变大在变小

    比如 让\alpha >1
    
    具体来说就是先扩大expansion convolution， 再做depthwise convolution，然后再缩小reduction convolution


- ### Linear BottleNecks
    在那里不需要relu，relu可能损失小于0的神经元

    本来就参数不多，学不到东西

此外，和ResNet不同的是，它有一种不跳阶的块：1x1Conv 然后Depthwise Conv 3x3 stride=2

## MobileNet V3
借鉴SENet，分支上用了一个全局的pooling

swish做激活函数

relu 6（到6截至）放在后面


## ShuffleNet V1
### 组卷积

对输入W x H x C的channel数分成g组， 对于每一组，有只供这一组用的k x k x C/g x N的卷积核，对这一组做N次普通卷积，（也就是C/g个平面卷积核对输入卷积再叠加，重复N次），得到N个输出，总共所有的组得到g x N个输出。

把每个组看成一个整体，这就像是深度卷积

分1组，就是传统卷积（相当于一个大核对所有的输入图进行卷积）

分组数和输入数等量：就是Depthwise Convolution

运算次数分析：用k x k x C/g x N x H x W得到一组的输出，要得到所有组的输出，k x k x C/g x N x H x W x g，也就是k x k x C x N x H x W，但是它的输出数量是普通卷积的g倍，所以相当于运算次数是普通卷积的1/g

这里的kernel一般是1x1的，为了通道可以叠加

得到的gxN的结果，通道和卷积核的相关性太高了，模型拟合能力差

需要加一个shuffle

![30](30.png)

## ShuffleNet V2
Gudie lines:

1. 内存访问消耗MAC（memory access cost）最小，当且仅当输入通道数=输出通道数
2. 组卷积会增加MAC
3. 分支增加也会增加MAC，降低平行度
4. 虽然卷积占运算的大头，对应元素的操作是不可以忽略的

## 轻量化框架对比
速度：

ShuffleNet-v2 > MolibeNet-V2 > ShuffleNet > MolibeNet > SqueezeNet

优先选ShuffleNet和MobileNet-v2

其他：
Xception resNexXt MobileID GhostNet

## 其他框架
feature pyramid network 金字塔

hourglass 沙漏模型 以图生图

Siamese 暹罗猫（孪生）网络，用于少样本的

## 

## 杂记
IMAGENET 有1400万张图片分类, 1000类

100万张用于检测, 定位, 框出来, 分类

2012年有Imagenet Large Scale Visaul Recognition Competition, AlexNet第一名

2014 ,这一比赛GoogLeNet冠军

2015 , ResNet冠军

resize 有很多方式， crop， 插值， 补充
### TODO 3通道变成N通道, 需要kxkx3xN的kernel, 中间层N个通道了, 然后再conv, 得到N'个通道需要多少个kernel?kxkxNxN'?
### 解答: 是的

### TODO VGG为什么pooling后能够size变小而channel变多？
### 解答： 因为做了卷积+stride操作
卷积核影响模型的大小（参数个数）和拟合效果

model size 只体现在参数上

FC是head，backbone是特征部分，而fc拉伸之后失去特征含义，属于功能层

### 实际应用

性能上来看，要变得更深

不考虑设备问题，先用Vgg或者Resnet做特征提取

再加功能层